In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import os

BASE = "/content/drive/MyDrive/AR_Stratified"

# THE ONLY FILES WE WILL USE — RAW YEARLY FILES
year_files = {
    2010: "hmi_2010_2010_fixed.csv",
    2011: "Sharp_Hmi_2011_2011_real1.csv", # Changed .cs to .csv
    2012: "hmi_2012_2012_fixed.csv",
    2013: "hmi_2013_2013_fixed.csv",
    2014: "hmi_2014_2014_fixed.csv",
    2015: "hmi_2015_2015_fixed.csv",
    2016: "hmi_2016_2016_fixed.csv",
    2017: "hmi_2017_2017_fixed.csv",
    2018: "hmi_2018_2018_fixed.csv",
    2019: "hmi_2019_2019_fixed.csv",
    2020: "hmi_2020_2020_fixed.csv",
    2021: "hmi_2021_2021_fixed.csv",
    2022: "hmi_2022_2022_fixed.csv",
    2023: "hmi_2023_2023_fixed.csv",
    2024: "hmi_2024_2024_fixed.csv",
    2025: "hmi_2025_2025_fixed.csv",
}

# -------------------------
# HELPERS
# -------------------------

def parse_trec(df):
    df["T_REC"] = df["T_REC"].astype(str).str.strip()
    try:
        df["T_REC_dt"] = pd.to_datetime(df["T_REC"],
                                        format="%Y.%m.%d_%H:%M:%S_TAI")
    except:
        df["T_REC_dt"] = pd.to_datetime(
            df["T_REC"].str.replace("_TAI", "", regex=False),
            errors="coerce"
        )
    df["year"] = df["T_REC_dt"].dt.year
    return df

def extract_primary_ar(x):
    s = str(x)
    if s.lower() in ["none", "nan", "missing", ""]:
        return np.nan
    m = re.search(r"\d+", s)
    return int(m.group()) if m else np.nan

def ensure_noaa(df):
    # Normalize column names by stripping whitespace
    df.columns = df.columns.str.strip()

    if "NOAA_AR" in df.columns:
        df["NOAA_AR"] = pd.to_numeric(df["NOAA_AR"],
                                      errors="coerce").astype("Int64")
    elif "NOAA_ARS" in df.columns:
        try:
            df["NOAA_AR"] = df["NOAA_ARS"].apply(extract_primary_ar).astype("Int64")
        except KeyError:
            # This should theoretically not be hit if "NOAA_ARS" in df.columns was True.
            # However, given the observed error, it provides robustness.
            print(f"Warning: Column 'NOAA_ARS' was checked to be present but caused a KeyError on access for year {df['year'].iloc[0] if 'year' in df.columns else 'unknown'}. Initializing NOAA_AR with NaN.")
            df["NOAA_AR"] = pd.Series([np.nan]*len(df), dtype="Int64")
    else:
        # Neither NOAA_AR nor NOAA_ARS found
        df["NOAA_AR"] = pd.Series([np.nan]*len(df), dtype="Int64")
    return df

# -------------------------
# CLEAN FROM SCRATCH
# -------------------------

all_years = []

print("=== CLEAN-SHEET BUILD STARTED ===")

for yr, fname in year_files.items():
    path = f"{BASE}/{fname}"

    print(f"\nLoading {yr}: {fname}")
    df = pd.read_csv(path)

    # 1. Parse time + year
    df = parse_trec(df)

    # 2. Extract NOAA_AR
    df = ensure_noaa(df)

    # 3. Keep only this year's rows (in case file spills)
    df = df[df["year"] == yr].copy()

    # 4. Remove duplicates (T_REC_dt + AR)
    df = df.drop_duplicates(subset=["T_REC_dt", "NOAA_AR"], keep="first")

    print(f"  Final {yr} shape:", df.shape)

    all_years.append(df)

# Combine all cleaned years
full = pd.concat(all_years, ignore_index=True)

print("\n=== AFTER MERGE ===")
print("Shape:", full.shape)
print("Years included:", sorted(full["year"].unique()))
print("\nCounts per year:")
print(full["year"].value_counts().sort_index())

# Save final clean file
OUT = f"{BASE}/HMI_AR_2010_2025_CLEAN_FRESH.csv"
full.to_csv(OUT, index=False)

print("\nSaved CLEAN-SHEET dataset to:")
print(OUT)

=== CLEAN-SHEET BUILD STARTED ===

Loading 2010: hmi_2010_2010_fixed.csv
  Final 2010 shape: (803, 30)

Loading 2011: Sharp_Hmi_2011_2011_real1.csv
  Final 2011 shape: (2195, 30)

Loading 2012: hmi_2012_2012_fixed.csv
  Final 2012 shape: (2114, 30)

Loading 2013: hmi_2013_2013_fixed.csv
  Final 2013 shape: (2618, 30)

Loading 2014: hmi_2014_2014_fixed.csv
  Final 2014 shape: (2362, 30)

Loading 2015: hmi_2015_2015_fixed.csv
  Final 2015 shape: (2196, 30)

Loading 2016: hmi_2016_2016_fixed.csv
  Final 2016 shape: (1453, 30)

Loading 2017: hmi_2017_2017_fixed.csv
  Final 2017 shape: (809, 30)

Loading 2018: hmi_2018_2018_fixed.csv
  Final 2018 shape: (335, 30)

Loading 2019: hmi_2019_2019_fixed.csv
  Final 2019 shape: (208, 30)

Loading 2020: hmi_2020_2020_fixed.csv
  Final 2020 shape: (448, 30)

Loading 2021: hmi_2021_2021_fixed.csv
  Final 2021 shape: (1183, 30)

Loading 2022: hmi_2022_2022_fixed.csv
  Final 2022 shape: (2320, 30)

Loading 2023: hmi_2023_2023_fixed.csv
  Final 2023 sha

In [ ]:
print("Counts per year:")
print(full["year"].value_counts().sort_index())

Counts per year:
year
2010     803
2011    2195
2012    2114
2013    2618
2014    2362
2015    2196
2016    1453
2017     809
2018     335
2019     208
2020     448
2021    1183
2022    2320
2023    2932
2024    2980
2025    1648
Name: count, dtype: int64


In [ ]:
import pandas as pd

df2024 = pd.read_csv("/content/drive/MyDrive/AR_Stratified/hmi_2024_2024_fixed.csv")
df2025 = pd.read_csv("/content/drive/MyDrive/AR_Stratified/hmi_2025_2025_fixed.csv")

print("2024 columns:", len(df2024.columns))
print(df2024.columns.tolist())

print("2025 columns:", len(df2025.columns))
print(df2025.columns.tolist())


2024 columns: 28
['T_REC', 'MEANGBZ', 'HARPNUM', 'R_VALUE', 'TOTUSJH', 'USFLUX', 'TOTPOT', 'MEANPOT', 'AREA_ACR', 'NOAA_ARS', 'LON_MIN', 'LON_MAX', 'LAT_MIN', 'LAT_MAX', 'QUALITY', 'USFLUX.1', 'MEANGAM', 'MEANGBT', 'MEANGBH', 'MEANJZD', 'TOTUSJZ', 'MEANALP', 'MEANJZH', 'ABSNJZH', 'SAVNCPP', 'MEANSHR', 'SHRGT45', 'AREA_ACR.1']
2025 columns: 27
['T_REC', 'MEANGBZ', 'HARPNUM', 'R_VALUE', 'TOTUSJH', 'USFLUX', 'TOTPOT', 'MEANPOT', 'AREA_ACR', 'NOAA_ARS', 'LON_MIN', 'LON_MAX', 'LAT_MIN', 'LAT_MAX', 'QUALITY', 'USFLUX.1', 'MEANGAM', 'MEANGBT', 'MEANGBH', 'MEANJZD', 'TOTUSJZ', 'MEANALP', 'MEANJZH', 'ABSNJZH', 'SAVNCPP', 'MEANSHR', 'SHRGT45']


In [ ]:
# Remove duplicate/useless columns
cols_to_drop = ["USFLUX.1", "AREA_ACR.1"]

for col in cols_to_drop:
    if col in full.columns:
        full = full.drop(columns=[col])

print("Final columns:", len(full.columns))
print(full.columns)


Final columns: 29
Index(['T_REC', 'MEANGBZ', 'HARPNUM', 'R_VALUE', 'TOTUSJH', 'USFLUX', 'TOTPOT',
       'MEANPOT', 'AREA_ACR', 'NOAA_ARS', 'LON_MIN', 'LON_MAX', 'LAT_MIN',
       'LAT_MAX', 'QUALITY', 'MEANGAM', 'MEANGBT', 'MEANGBH', 'MEANJZD',
       'TOTUSJZ', 'MEANALP', 'MEANJZH', 'ABSNJZH', 'SAVNCPP', 'MEANSHR',
       'SHRGT45', 'T_REC_dt', 'year', 'NOAA_AR'],
      dtype='object')


In [ ]:
# =============================================================
# 1) BASE SETUP
# =============================================================
BASE = "/content/drive/MyDrive/AR_Stratified"

# Load the CLEAN-SHEET merged dataset you built earlier
full = pd.read_csv(f"{BASE}/HMI_AR_2010_2025_CLEAN_FRESH.csv")
print("Loaded CLEAN FRESH dataset:", full.shape)
print("Columns:", list(full.columns))


Loaded CLEAN FRESH dataset: (26604, 31)
Columns: ['T_REC', 'MEANGBZ', 'HARPNUM', 'R_VALUE', 'TOTUSJH', 'USFLUX', 'TOTPOT', 'MEANPOT', 'AREA_ACR', 'NOAA_ARS', 'LON_MIN', 'LON_MAX', 'LAT_MIN', 'LAT_MAX', 'QUALITY', 'USFLUX.1', 'MEANGAM', 'MEANGBT', 'MEANGBH', 'MEANJZD', 'TOTUSJZ', 'MEANALP', 'MEANJZH', 'ABSNJZH', 'SAVNCPP', 'MEANSHR', 'SHRGT45', 'T_REC_dt', 'year', 'NOAA_AR', 'AREA_ACR.1']


In [ ]:
# =============================================================
# 2) CORE 16 SHARP FEATURES + TIME + AR ID
# =============================================================

core_features = [
    # Time + AR identity
    "T_REC_dt", "year", "NOAA_AR",

    # 16 physics-driven AR features
    "MEANGBZ", "MEANGAM", "MEANGBT", "MEANGBH",
    "MEANJZD", "TOTUSJZ", "MEANALP", "MEANJZH",
    "ABSNJZH", "SAVNCPP", "MEANSHR", "SHRGT45",
    "R_VALUE", "USFLUX", "TOTPOT", "TOTUSJH"
]

# Extract only the features available in the dataset
available_core = [c for c in core_features if c in full.columns]
missing_core = [c for c in core_features if c not in full.columns]

print("Available CORE features:", available_core)
print("Missing CORE features:", missing_core)

ml16 = full[available_core].copy()

# Save file
path16 = f"{BASE}/HMI_AR_2010_2025_ML_READY_16.csv"
ml16.to_csv(path16, index=False)

print("\n=== ML-READY-16 DATASET ===")
print("Shape:", ml16.shape)
print("Saved to:", path16)


Available CORE features: ['T_REC_dt', 'year', 'NOAA_AR', 'MEANGBZ', 'MEANGAM', 'MEANGBT', 'MEANGBH', 'MEANJZD', 'TOTUSJZ', 'MEANALP', 'MEANJZH', 'ABSNJZH', 'SAVNCPP', 'MEANSHR', 'SHRGT45', 'R_VALUE', 'USFLUX', 'TOTPOT', 'TOTUSJH']
Missing CORE features: []

=== ML-READY-16 DATASET ===
Shape: (26604, 19)
Saved to: /content/drive/MyDrive/AR_Stratified/HMI_AR_2010_2025_ML_READY_16.csv


In [ ]:
# =============================================================
# 3) FULL 29-FEATURE ADVANCED DATASET
# =============================================================

# Select safe columns (everything except duplicates)
drop_cols = ["USFLUX.1", "AREA_ACR.1"]  # Already removed earlier

full29 = full[[c for c in full.columns if c not in drop_cols]].copy()

path29 = f"{BASE}/HMI_AR_2010_2025_ML_READY_29.csv"
full29.to_csv(path29, index=False)

print("\n=== ML-READY-29 DATASET ===")
print("Shape:", full29.shape)
print("Saved to:", path29)



=== ML-READY-29 DATASET ===
Shape: (26604, 29)
Saved to: /content/drive/MyDrive/AR_Stratified/HMI_AR_2010_2025_ML_READY_29.csv


In [ ]:
print("\nYEAR COUNTS (16-feature dataset):")
print(ml16["year"].value_counts().sort_index())

print("\nYEAR COUNTS (29-feature dataset):")
print(full29["year"].value_counts().sort_index())

print("\nAny duplicates (T_REC_dt + NOAA_AR)?")
print(full29.duplicated(subset=["T_REC_dt", "NOAA_AR"]).sum())



YEAR COUNTS (16-feature dataset):
year
2010     803
2011    2195
2012    2114
2013    2618
2014    2362
2015    2196
2016    1453
2017     809
2018     335
2019     208
2020     448
2021    1183
2022    2320
2023    2932
2024    2980
2025    1648
Name: count, dtype: int64

YEAR COUNTS (29-feature dataset):
year
2010     803
2011    2195
2012    2114
2013    2618
2014    2362
2015    2196
2016    1453
2017     809
2018     335
2019     208
2020     448
2021    1183
2022    2320
2023    2932
2024    2980
2025    1648
Name: count, dtype: int64

Any duplicates (T_REC_dt + NOAA_AR)?
0


In [ ]:
print("\nAny duplicates (T_REC_dt + NOAA_AR)?")
print(full29.duplicated(subset=["T_REC_dt", "NOAA_AR"]).sum())



Any duplicates (T_REC_dt + NOAA_AR)?
0


# **PROCESSING OF THE GOES DATA**

In [ ]:
import pandas as pd
import numpy as np
import glob
import re

BASE = "/content/drive/MyDrive/AR_Stratified"

goes_files = sorted(glob.glob(f"{BASE}/GOES XRAY events for *.txt"))
print("FILES FOUND:", goes_files)

rows = []

def parse_goes_line(line):
    """
    Robust parser for one GOES flare line.
    Returns dict or None if the line is not a flare record.
    """
    line = line.strip()

    # Skip empty / metadata / comments
    if not line:
        return None
    if "Written" in line or "GOES" in line or "events" in line.lower():
        return None

    tokens = line.split()
    if len(tokens) < 4:
        return None

    # First token must be a date like 3-Jan-2020
    if not re.match(r"^\d{1,2}-[A-Za-z]{3}-\d{4}$", tokens[0]):
        return None

    date = tokens[0]

    # Find index of flare class (A8.9, B1.0, C2.5, M3.1, X1.0)
    class_idx = None
    for i, t in enumerate(tokens):
        if re.match(r"^[A-Z][0-9](\.[0-9])?$", t):
            class_idx = i
            break
    if class_idx is None:
        return None

    # Times: assume tokens[1] = start, tokens[2] = peak if exists
    start_time = tokens[1] if len(tokens) > 1 else None
    peak_time = tokens[2] if len(tokens) > 2 else None

    flare_class = tokens[class_idx]

    # Location (if present) is next token after class
    location = tokens[class_idx + 1] if class_idx + 1 < len(tokens) else None

    # Search for a 4–5 digit NOAA AR number in the tail
    NOAA_AR = np.nan
    for t in tokens[class_idx + 1:]:
        if re.fullmatch(r"\d{4,5}", t):
            NOAA_AR = int(t)
            break

    return {
        "flare_date": date,
        "start_time": start_time,
        "peak_time": peak_time,
        "flare_class": flare_class,
        "location": location,
        "NOAA_AR": NOAA_AR,
    }

# =============================================================
# Parse all GOES files line by line
# =============================================================

for file in goes_files:
    print(f"\nProcessing: {file}")
    with open(file, "r", errors="ignore") as f:
        for line in f:
            parsed = parse_goes_line(line)
            if parsed is not None:
                rows.append(parsed)

goes_df = pd.DataFrame(rows)

print("\nAFTER PARSING ALL FILES:")
print("Shape:", goes_df.shape)
print(goes_df.head(15))

# Build datetime column (only if we have data)
if not goes_df.empty:
    goes_df["flare_start_dt"] = pd.to_datetime(
        goes_df["flare_date"] + " " + goes_df["start_time"],
        errors="coerce"
    )
    goes_df["year"] = goes_df["flare_start_dt"].dt.year

    print("\nYears in GOES data:", goes_df["year"].value_counts().sort_index())
else:
    print("\n⚠️ goes_df is empty – parser matched nothing.")


FILES FOUND: ['/content/drive/MyDrive/AR_Stratified/GOES XRAY events for 2010.txt', '/content/drive/MyDrive/AR_Stratified/GOES XRAY events for 2011.txt', '/content/drive/MyDrive/AR_Stratified/GOES XRAY events for 2012.txt', '/content/drive/MyDrive/AR_Stratified/GOES XRAY events for 2013.txt', '/content/drive/MyDrive/AR_Stratified/GOES XRAY events for 2014.txt', '/content/drive/MyDrive/AR_Stratified/GOES XRAY events for 2015.txt', '/content/drive/MyDrive/AR_Stratified/GOES XRAY events for 2016.txt', '/content/drive/MyDrive/AR_Stratified/GOES XRAY events for 2017.txt', '/content/drive/MyDrive/AR_Stratified/GOES XRAY events for 2018.txt', '/content/drive/MyDrive/AR_Stratified/GOES XRAY events for 2019.txt', '/content/drive/MyDrive/AR_Stratified/GOES XRAY events for 2020.txt', '/content/drive/MyDrive/AR_Stratified/GOES XRAY events for 2021.txt', '/content/drive/MyDrive/AR_Stratified/GOES XRAY events for 2022.txt', '/content/drive/MyDrive/AR_Stratified/GOES XRAY events for 2023.txt', '/cont

# **STEP 1 – Clean GOES & keep only M/X flares with known AR**

In [ ]:
import pandas as pd
import numpy as np

BASE = "/content/drive/MyDrive/AR_Stratified"

# If you still have goes_df from previous cell, reuse it.
# Otherwise reload from backup CSV (optional):
# goes_df = pd.read_csv(f"{BASE}/GOES_ALL_PARSED.csv", parse_dates=["flare_start_dt"])

print("GOES raw shape:", goes_df.shape)

# Drop rows without a valid flare_start_dt
goes_df = goes_df.dropna(subset=["flare_date", "start_time"])
goes_df["flare_start_dt"] = pd.to_datetime(
    goes_df["flare_date"] + " " + goes_df["start_time"],
    errors="coerce"
)
goes_df = goes_df.dropna(subset=["flare_start_dt"])

# Standardise flare class
goes_df["flare_class"] = goes_df["flare_class"].astype(str).str.strip()

# Split letter (A/B/C/M/X) and magnitude (e.g. 1.2, 5.6)
goes_df["class_letter"] = goes_df["flare_class"].str[0]
goes_df["class_mag"] = pd.to_numeric(goes_df["flare_class"].str[1:],
                                     errors="coerce")

# Keep only rows with a valid AR number & valid class
goes_df["NOAA_AR"] = pd.to_numeric(goes_df["NOAA_AR"], errors="coerce").astype("Int64")
goes_df = goes_df.dropna(subset=["NOAA_AR", "class_letter"])

# Filter to M & X flares (for now – strongest events)
goes_mx = goes_df[goes_df["class_letter"].isin(["M", "X"])].copy()

goes_mx["year"] = goes_mx["flare_start_dt"].dt.year

print("\nM+X FLARES ONLY:")
print("Shape:", goes_mx.shape)
print(goes_mx.head(10))

print("\nM+X counts by year:")
print(goes_mx["year"].value_counts().sort_index())


GOES raw shape: (34723, 8)

M+X FLARES ONLY:
Shape: (2240, 10)
      flare_date start_time peak_time flare_class location  NOAA_AR  \
102  20-Jan-2010      10:46     10:59        M1.8   S25E87    11041   
166   7-Feb-2010      02:20     02:34        M6.4   N21E11    11045   
192   8-Feb-2010      11:57     12:03        M1.1   N21W07    11045   
234  12-Feb-2010      11:19     11:26        M8.3   N33E13    11046   
238  12-Feb-2010      17:52     18:08        M1.1   N33E10    11046   
517  12-Jun-2010      00:30     00:57        M2.0   N23W47    11081   
530  13-Jun-2010      05:30     05:39        M1.0   S24W82    11079   
531  13-Jun-2010      05:30     05:39        M1.0   S24W82    11079   
710   7-Aug-2010      17:48     18:24        M1.0   N14E37    11093   
711   7-Aug-2010      17:55     18:24        M1.0   N14E37    11093   

         flare_start_dt  year class_letter  class_mag  
102 2010-01-20 10:46:00  2010            M        1.8  
166 2010-02-07 02:20:00  2010            M 

# S**TEP 2 – Load AR dataset & attach 3-day preflare labels**

We’ll use your 16-feature core AR dataset and add a binary label:
label_MX_3d = 1 → this AR snapshot is within 3 days before an M/X flare from that AR
label_MX_3d = 0 → otherwise

In [ ]:
# ==========================================
# Load AR 16-feature dataset
# ==========================================
ar = pd.read_csv(f"{BASE}/HMI_AR_2010_2025_ML_READY_16.csv")

print("AR shape:", ar.shape)
print("AR columns:", list(ar.columns))

# Ensure datetime
ar["T_REC_dt"] = pd.to_datetime(ar["T_REC_dt"], errors="coerce")
ar["year"] = pd.to_numeric(ar["year"], errors="coerce").astype("Int64")
ar["NOAA_AR"] = pd.to_numeric(ar["NOAA_AR"], errors="coerce").astype("Int64")

# Drop AR rows with missing time or AR id
ar = ar.dropna(subset=["T_REC_dt", "NOAA_AR"]).copy()
print("\nAR after dropping missing T_REC_dt/NOAA_AR:", ar.shape)

# Initialise label
ar["label_MX_3d"] = 0

# (Optional) store if it was followed by X vs M flare
ar["future_max_class"] = pd.Series(index=ar.index, dtype="object")

# Sort AR and GOES by time for sanity
ar = ar.sort_values(["NOAA_AR", "T_REC_dt"]).reset_index(drop=True)
goes_mx = goes_mx.sort_values(["NOAA_AR", "flare_start_dt"]).reset_index(drop=True)

# ==========================================
# Attach 3-day preflare labels
# ==========================================
from tqdm import tqdm  # if not installed, comment out or install

print("\nLabelling AR rows with 3-day preflare window for M/X flares...")

for idx, row in tqdm(goes_mx.iterrows(), total=len(goes_mx)):
    ar_num = row["NOAA_AR"]
    flare_time = row["flare_start_dt"]
    flare_cls = row["flare_class"]

    # 3-day window before flare (inclusive)
    start_win = flare_time - pd.Timedelta(days=3)
    end_win   = flare_time

    mask = (
        (ar["NOAA_AR"] == ar_num) &
        (ar["T_REC_dt"] >= start_win) &
        (ar["T_REC_dt"] <= end_win)
    )

    if mask.any():
        ar.loc[mask, "label_MX_3d"] = 1

        # Optionally track most severe flare class seen
        # (e.g. if both M and X, we keep X)
        current = ar.loc[mask, "future_max_class"]
        new_vals = flare_cls

        # If existing is NaN or lower class, overwrite
        # (Simple logic: X > M; within same letter keep higher magnitude)
        def choose_best(old, new):
            if pd.isna(old):
                return new
            # Compare by class letter priority
            if old[0] == new[0]:
                try:
                    return old if float(old[1:]) >= float(new[1:]) else new
                except:
                    return old
            # X > M > C etc.
            order = {"X": 3, "M": 2, "C": 1, "B": 0, "A": 0}
            return old if order.get(old[0], 0) >= order.get(new[0], 0) else new

        ar.loc[mask, "future_max_class"] = [
            choose_best(o, new_vals) for o in current
        ]

print("\nLabel distribution (label_MX_3d):")
print(ar["label_MX_3d"].value_counts())


AR shape: (26604, 19)
AR columns: ['T_REC_dt', 'year', 'NOAA_AR', 'MEANGBZ', 'MEANGAM', 'MEANGBT', 'MEANGBH', 'MEANJZD', 'TOTUSJZ', 'MEANALP', 'MEANJZH', 'ABSNJZH', 'SAVNCPP', 'MEANSHR', 'SHRGT45', 'R_VALUE', 'USFLUX', 'TOTPOT', 'TOTUSJH']

AR after dropping missing T_REC_dt/NOAA_AR: (22355, 19)

Labelling AR rows with 3-day preflare window for M/X flares...


100%|██████████| 2240/2240 [00:05<00:00, 402.18it/s]


Label distribution (label_MX_3d):
label_MX_3d
0    20888
1     1467
Name: count, dtype: int64


In [ ]:
out_path = f"{BASE}/HMI_AR_2010_2025_ML_READY_16_LABELED_MX3D.csv"
ar.to_csv(out_path, index=False)
print("\nSaved labeled AR dataset to:")
print(out_path)



Saved labeled AR dataset to:
/content/drive/MyDrive/AR_Stratified/HMI_AR_2010_2025_ML_READY_16_LABELED_MX3D.csv


# **🔥 1. Prepare full GOES (all classes A–X, with AR + time)**

In [ ]:
import numpy as np
import pandas as pd

# Start from full GOES (not just M/X)
goes_all = goes_df.copy()

# Ensure datetime & clean fields
goes_all = goes_all.dropna(subset=["flare_date", "start_time"])
goes_all["flare_start_dt"] = pd.to_datetime(
    goes_all["flare_date"] + " " + goes_all["start_time"],
    errors="coerce"
)
goes_all = goes_all.dropna(subset=["flare_start_dt"])

goes_all["flare_class"] = goes_all["flare_class"].astype(str).str.strip()
goes_all["class_letter"] = goes_all["flare_class"].str[0]
goes_all["class_mag"] = pd.to_numeric(goes_all["flare_class"].str[1:], errors="coerce")

goes_all["NOAA_AR"] = pd.to_numeric(goes_all["NOAA_AR"], errors="coerce").astype("Int64")
goes_all = goes_all.dropna(subset=["NOAA_AR", "class_letter"])

goes_all["year"] = goes_all["flare_start_dt"].dt.year

print("GOES (all classes) shape:", goes_all.shape)
print(goes_all.head(10))


GOES (all classes) shape: (27667, 10)
    flare_date start_time peak_time flare_class location  NOAA_AR  \
0   1-Jan-2010      12:02     12:09        B1.9   S29W31    11039   
1   1-Jan-2010      12:33     12:43        B2.3   S29W31    11039   
2   1-Jan-2010      23:29     23:33        B1.1   S25W39    11039   
3   2-Jan-2010      03:10     03:13        B1.1   S26W39    11039   
5   2-Jan-2010      07:58     08:05        B6.4   S25W38    11039   
8   2-Jan-2010      14:08     14:16        C2.6   S28W40    11039   
14  3-Jan-2010      07:43     07:46        B1.5   S27W49    11039   
16  3-Jan-2010      10:50     11:03        B1.5   S28W56    11039   
21  3-Jan-2010      15:52     16:03        B6.7   S28W57    11039   
29  5-Jan-2010      08:46     08:52        B1.2   S28W81    11039   

        flare_start_dt  year class_letter  class_mag  
0  2010-01-01 12:02:00  2010            B        1.9  
1  2010-01-01 12:33:00  2010            B        2.3  
2  2010-01-01 23:29:00  2010         

# **2. Copy AR data and add multi-class labels (A/B/C/M/X + times)**

We’ll:

Keep your existing ar (with label_MX_3d)

Make a new copy called ar_all

Add:

label_any_3d → 0/1 (any flare A–X within 3 days)

future_max_any_class → strongest flare class (A/B/C/M/X + magnitude) in next 3 days

future_flare_start_dt_any → datetime of that flare

future_flare_peak_time_any → peak time string

Strength order:

X > M > C > B > A; within same letter, higher magnitude wins (e.g. M6.4 > M1.1).

In [ ]:
# Work on a fresh copy so we don't disturb the previous labelled dataset
ar_all = ar.copy()

# Initialise new labels
ar_all["label_any_3d"] = 0
ar_all["future_max_any_class"] = pd.Series(index=ar_all.index, dtype="object")
ar_all["future_flare_start_dt_any"] = pd.NaT
ar_all["future_flare_peak_time_any"] = pd.Series(index=ar_all.index, dtype="object")

# Ensure types
ar_all["T_REC_dt"] = pd.to_datetime(ar_all["T_REC_dt"], errors="coerce")
ar_all["NOAA_AR"] = pd.to_numeric(ar_all["NOAA_AR"], errors="coerce").astype("Int64")
ar_all = ar_all.dropna(subset=["T_REC_dt", "NOAA_AR"]).copy()

goes_all = goes_all.sort_values(["NOAA_AR", "flare_start_dt"]).reset_index(drop=True)
ar_all = ar_all.sort_values(["NOAA_AR", "T_REC_dt"]).reset_index(drop=True)

# Helper: convert class string like 'M6.4' to numeric score
def flare_score(cls_str):
    if pd.isna(cls_str):
        return -1
    s = str(cls_str).strip()
    if len(s) < 2:
        return -1
    letter = s[0]
    try:
        mag = float(s[1:])
    except Exception:
        mag = 0.0
    order = {"X": 5, "M": 4, "C": 3, "B": 2, "A": 1}
    return order.get(letter, 0) * 100 + mag

print("\nLabelling AR rows with 3-day preflare window for ANY flare class (A–X)...")

for _, f in goes_all.iterrows():
    ar_num = f["NOAA_AR"]
    flare_time = f["flare_start_dt"]
    flare_cls = f["flare_class"]
    peak_time = f.get("peak_time", None)

    # 3-day preflare window
    start_win = flare_time - pd.Timedelta(days=3)
    end_win   = flare_time

    mask = (
        (ar_all["NOAA_AR"] == ar_num) &
        (ar_all["T_REC_dt"] >= start_win) &
        (ar_all["T_REC_dt"] <= end_win)
    )

    if not mask.any():
        continue

    new_score = flare_score(flare_cls)
    idxs = ar_all.index[mask]

    for idx in idxs:
        old_cls = ar_all.at[idx, "future_max_any_class"]
        old_score = flare_score(old_cls)

        # If new flare stronger than any already assigned, overwrite
        if new_score > old_score:
            ar_all.at[idx, "label_any_3d"] = 1
            ar_all.at[idx, "future_max_any_class"] = flare_cls
            ar_all.at[idx, "future_flare_start_dt_any"] = flare_time
            ar_all.at[idx, "future_flare_peak_time_any"] = peak_time

print("\nLabel distribution (label_any_3d):")
print(ar_all["label_any_3d"].value_counts())

print("\nTop future_max_any_class values:")
print(ar_all["future_max_any_class"].value_counts().head(20))



Labelling AR rows with 3-day preflare window for ANY flare class (A–X)...

Label distribution (label_any_3d):
label_any_3d
0    13144
1     9211
Name: count, dtype: int64

Top future_max_any_class values:
future_max_any_class
C1.1    191
C1.2    184
C1.0    179
C1.5    177
C1.7    176
C1.4    173
C1.6    168
C1.3    153
C2.2    149
C1.9    148
C1.8    144
C2.3    136
C3.0    129
M1.1    124
C2.1    117
M1.0    116
C2.0    116
C2.9    108
C2.6    106
C2.4     98
Name: count, dtype: int64


In [ ]:
out_path_all = f"{BASE}/HMI_AR_2010_2025_ML_READY_16_LABELED_ALLCLASS_3D.csv"
ar_all.to_csv(out_path_all, index=False)
print("\nSaved multi-class (A–X) labelled dataset to:")
print(out_path_all)



Saved multi-class (A–X) labelled dataset to:
/content/drive/MyDrive/AR_Stratified/HMI_AR_2010_2025_ML_READY_16_LABELED_ALLCLASS_3D.csv
